In [2]:
# -*- coding: utf-8 -*-

# don
import pandas as pd
import numpy as np
import librosa
import os
import zipfile
import csv
import shutil
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from concurrent.futures import ThreadPoolExecutor, as_completed
from sklearn.metrics import (
    roc_auc_score,        
)


In [3]:
"""
Script for extracting data from https://figshare.com/articles/dataset/Sounds_of_the_Eleutherodactylus_frog_community_from_Puerto_Rico/806302?file=3104183
Unzips all the zips from root
Simply Unzip downloaded file and provide path root of folder
"""


def extract_zip_file(file_path, extract_to):
    """
    Extracts a single zip file to a specified directory.

    Args:
        file_path (str): Path to the zip file.
        extract_to (str): Path to the directory where the zip file will be extracted.
    """
    with zipfile.ZipFile(file_path, "r") as zip_ref:
        zip_ref.extractall(extract_to)
        # print(f"{os.path.basename(file_path)} extracted to {os.path.abspath(extract_to)}")


def extract_zip_files(zip_folder, extract_to):
    """
    Extracts all zip files from a folder to a specified directory using threading.

    Args:
        zip_folder (str): Path to the folder containing zip files.
        extract_to (str): Path to the directory where zip files will be extracted.
    """
    # Make sure the extraction directory exists
    os.makedirs(extract_to, exist_ok=True)

    data_file = "FrequencyRange_by_species_and_site_Averages.csv"
    shutil.copyfile(os.path.join(zip_folder, data_file) , os.path.join(extract_to, data_file))

    # List all zip files in the folder
    zip_files = [
        os.path.join(zip_folder, item)
        for item in os.listdir(zip_folder)
        if item.endswith(".zip")
    ]

    # Use ThreadPoolExecutor to extract zip files concurrently
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(extract_zip_file, zip_file, extract_to)
            for zip_file in zip_files
        ]

        # Wait for all futures to complete
        for future in futures:
            future.result()
    
    print("Done.")

In [4]:
filepath = "/home/alo/Downloads/datos/806302/"
ExtractTo = "/home/alo/Downloads/datos/Extracted"
extract_zip_files(filepath, ExtractTo)

Done.


In [5]:
def readAveragesData(path: str):

    data = []
    # Read the CSV file and store the data in a list of dictionaries
    with open(path, "r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            data.append(row)

    return data


def prepare_csv(data_dir, output) -> None:

    
    averagesData = readAveragesData(
        os.path.join(data_dir, "FrequencyRange_by_species_and_site_Averages.csv")
    )

    data = []

    # Iterate through each subfolder
    for siteDataSet in os.listdir(data_dir):
        site_folder = os.path.join(data_dir, siteDataSet)
        if os.path.isdir(site_folder):
            # example siteId  "Site01-1" such that the 4-6 index represents the site id; in this case 01
            siteId = int(siteDataSet[4:6])
            SiteData = [
                averageClassification
                for averageClassification in averagesData
                if int(averageClassification["SiteID"]) == siteId
            ]

            classifications = ", ".join(
                [classification["Species"] for classification in SiteData]
            )
            for audio_recording in os.listdir(site_folder):
                if audio_recording.endswith(".wav"):
                    audio_recording_abs_path = os.path.abspath(
                        os.path.join(site_folder, audio_recording)
                    )

                    data.append([siteId, audio_recording_abs_path, classifications])

    # Create DataFrame
    df = pd.DataFrame(
        data,
        columns=[
            "siteId",
            "filename",
            "species",
        ],
    )

    df.to_csv(output, index=False)

In [10]:
data_dir = ExtractTo
output = "processed/processed.csv"
prepare_csv(data_dir, output)

In [11]:
def extract_features(file_path):
    """
    Extract features from audio file using librosa.

    Args:
        file_path (str): Path to the audio file.

    Returns:
        np.array: Extracted features.
    """
    audio, sr = librosa.load(file_path)
    result = np.array([])

    # MFCC
    mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sr).T, axis=0)
    result = np.hstack((result, mfccs))

    # Chroma
    stft = np.abs(librosa.stft(audio))
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sr).T, axis=0)
    result = np.hstack((result, chroma))

    # Mel-scaled spectrogram
    mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sr).T, axis=0)
    result = np.hstack((result, mel))

    return result


def process_data(data_csv_path):
    """Read Csv with fileanme and generate spectrogram for each sample

    Returns:
        DataFrame: dataframe with all data
    """
    # data_csv_path = sys.argv[1]

    df = pd.read_csv(data_csv_path)

    # Initialize a list to store the results
    spectrograms = []

    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(extract_features, row["filename"])
            for _, row in df.iterrows()
        ]

        for future in as_completed(futures):
            try:
                spectrogram = future.result()
                spectrograms.append(spectrogram)
            except Exception as exc:
                print(f"Generated an exception: {exc}")

    # Convert the list of spectrograms into a DataFrame
    spectrogram_df = pd.DataFrame(spectrograms)

    # Concatenate the original DataFrame with the new DataFrame containing spectrograms
    df = pd.concat([df, spectrogram_df], axis=1)

    return df

In [33]:
csv_dir = output
df = process_data(csv_dir)
df.head()

,siteId,filename,species,0,1,2,3,4,5,6,...,150,151,152,153,154,155,156,157,158,159
0,12,/home/alo/Downloads/datos/Extracted/Site12-2/L...,"E. coqui - co, E. coqui - qui, E. wightmanae",-306.485260,115.118217,-57.712772,28.945372,33.879120,-15.323828,4.970051,...,0.000012,0.000007,0.000008,0.000008,0.000007,0.000004,0.000003,0.000002,1.669141e-06,3.145811e-07
1,12,/home/alo/Downloads/datos/Extracted/Site12-2/L...,"E. coqui - co, E. coqui - qui, E. wightmanae",-357.852692,109.087685,-93.553185,46.544666,37.265129,3.004175,1.793162,...,0.000010,0.000004,0.000003,0.000002,0.000002,0.000002,0.000002,0.000002,7.716953e-07,6.098661e-08
2,12,/home/alo/Downloads/datos/Extracted/Site12-2/L...,"E. coqui - co, E. coqui - qui, E. wightmanae",-323.070892,112.686317,-91.024620,42.821640,43.136612,-16.130556,7.841372,...,0.000009,0.000003,0.000003,0.000003,0.000003,0.000002,0.000001,0.000001,5.145531e-07,4.375238e-08
3,12,/home/alo/Downloads/datos/Extracted/Site12-2/L...,"E. coqui - co, E. coqui - qui, E. wightmanae",-326.180695,109.096848,-49.765556,10.103225,51.272846,1.646845,-11.518492,...,0.000021,0.000020,0.000015,0.000009,0.000006,0.000004,0.000015,0.000024,1.250767e-05,1.589573e-06
4,12,/home/alo/Downloads/datos/Extracted/Site12-2/L...,"E. coqui - co, E. coqui - qui, E. wightmanae",-311.847443,124.398911,-87.386368,44.579063,32.205944,-15.415882,7.980429,...,0.000009,0.000003,0.000003,0.000002,0.000002,0.000002,0.000001,0.000001,5.204484e-07,4.127714e-08


In [34]:
x = df.drop(
    columns=["filename", "species"]
)  # Adjust this to include only feature columns
# Convert all column names to strings
x.columns = x.columns.astype(str)

x

,siteId,0,1,2,3,4,5,6,7,8,...,150,151,152,153,154,155,156,157,158,159
0,12,-306.485260,115.118217,-57.712772,28.945372,33.879120,-15.323828,4.970051,5.750704,-5.704476,...,0.000012,0.000007,0.000008,0.000008,0.000007,0.000004,0.000003,0.000002,1.669141e-06,3.145811e-07
1,12,-357.852692,109.087685,-93.553185,46.544666,37.265129,3.004175,1.793162,6.590044,-6.446641,...,0.000010,0.000004,0.000003,0.000002,0.000002,0.000002,0.000002,0.000002,7.716953e-07,6.098661e-08
2,12,-323.070892,112.686317,-91.024620,42.821640,43.136612,-16.130556,7.841372,11.723942,-11.795463,...,0.000009,0.000003,0.000003,0.000003,0.000003,0.000002,0.000001,0.000001,5.145531e-07,4.375238e-08
3,12,-326.180695,109.096848,-49.765556,10.103225,51.272846,1.646845,-11.518492,17.419624,3.736421,...,0.000021,0.000020,0.000015,0.000009,0.000006,0.000004,0.000015,0.000024,1.250767e-05,1.589573e-06
4,12,-311.847443,124.398911,-87.386368,44.579063,32.205944,-15.415882,7.980429,10.756527,-9.538747,...,0.000009,0.000003,0.000003,0.000002,0.000002,0.000002,0.000001,0.000001,5.204484e-07,4.127714e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545,4,-272.171631,106.364799,-71.935768,3.470784,64.135635,45.543198,-13.712868,11.404665,20.267740,...,0.000019,0.000012,0.000010,0.000010,0.000014,0.000024,0.000028,0.000024,9.445997e-06,7.249362e-07
1546,4,-301.928741,113.540588,-88.269318,0.868106,73.557594,43.375679,-12.706711,14.790010,11.994915,...,0.000014,0.000008,0.000005,0.000003,0.000003,0.000004,0.000005,0.000006,8.108656e-06,1.501757e-06
1547,4,-333.100372,116.606209,-57.372360,8.567136,52.794895,33.931950,-3.181213,8.148308,20.404114,...,0.000010,0.000005,0.000004,0.000004,0.000004,0.000004,0.000006,0.000005,3.474788e-06,2.086388e-07
1548,4,-316.301086,147.537399,-36.467831,24.430267,19.927158,21.474241,-6.610122,-0.935251,17.519373,...,0.000010,0.000005,0.000004,0.000004,0.000003,0.000003,0.000003,0.000004,1.798972e-06,1.822809e-07


In [35]:
y = df["species"]

# Encode the target labels as integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y



array([10, 10, 10, ...,  2,  2,  2])

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

classifier = RandomForestClassifier(n_estimators=600, max_depth=18, min_samples_leaf=3)

classifier.fit(x_train, y_train)

y_pred = classifier.predict_proba(
    x_test,
)

accuracy = roc_auc_score(y_test, y_pred, multi_class="ovr")
print("Accuracy :", accuracy)




Accuracy : 0.9962154033143414
